<a href="https://colab.research.google.com/github/saerarawas/AAI614_rawas/blob/main/Week%207/Pandas_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import glob

# Use glob to find all matching files
file_pattern = '/content/nyc-parking-tickets/*2017.csv'
files = glob.glob(file_pattern)

# Start timing
start_time = time.time()

# Load the dataset, handling inconsistent number of columns
df_list = []
for file in files:
    try:
        df = pd.read_csv(file)  # Attempt to read the file normally
        #df = pd.read_csv(file, dtype=dtype, na_values=['', 'NaN'], keep_default_na=True)
        df_list.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file {file}: {e}")
        # Handle the error, e.g., skip the file or use 'on_bad_lines' parameter
        # Instead of skipping the file entirely, try to read it with error handling:
        df = pd.read_csv(file, on_bad_lines='skip')  # Skip problematic lines
        df_list.append(df) # Append the dataframe to the list even if it has errors

df_pandas = pd.concat(df_list, ignore_index=True)

# End timing
end_time = time.time()
print(f"Pandas Load Data - Wall Time: {end_time - start_time} seconds")

# Check the first few rows of the dataframe
print(df_pandas.head())

Pandas Load Data - Wall Time: 0.48729681968688965 seconds
   Summons Number Plate ID Registration State Plate Type  Issue Date  \
0      5092469481  GZH7067                 NY        PAS  07/10/2016   
1      5092451658  GZH7067                 NY        PAS  07/08/2016   
2      4006265037  FZX9232                 NY        PAS  08/23/2016   
3      8478629828  66623ME                 NY        COM  06/14/2017   
4      7868300310  37033JV                 NY        COM  11/21/2016   

   Violation Code Vehicle Body Type Vehicle Make Issuing Agency  Street Code1  \
0               7              SUBN        TOYOT              V             0   
1               7              SUBN        TOYOT              V             0   
2               5              SUBN         FORD              V             0   
3              47              REFG        MITSU              T         10610   
4              69              DELV        INTER              T         10510   

   ...  Vehicle Color 

In [19]:
# Start timing
start_time = time.time()

# Calculate missing values
missing_values = df_pandas.isnull().sum()

# Calculate percentage of missing values
missing_count_pct = (missing_values / len(df_pandas)) * 100

# End timing
end_time = time.time()
print(f"Pandas Calculate Missing Values - Wall Time: {end_time - start_time} seconds")



Pandas Calculate Missing Values - Wall Time: 0.08165788650512695 seconds


In [20]:
# Start timing
start_time = time.time()

# Identify columns to drop
columns_to_drop = missing_count_pct[missing_count_pct > 60].index

# Drop the columns
df_dropped_pandas = df_pandas.drop(columns=columns_to_drop)

# End timing
end_time = time.time()
print(f"Pandas Drop Columns - Wall Time: {end_time - start_time} seconds")



Pandas Drop Columns - Wall Time: 0.013362646102905273 seconds


In [26]:
import dask.dataframe as dd
import time

# Define the file pattern to load all files matching the criteria
file_pattern = '/content/nyc-parking-tickets/*2017.csv'

# Specify data types for problematic columns
dtype_spec = {
    'House Number': 'object',
    'Time First Observed': 'object'
}

# Start timing
start_time = time.time()

# Load the dataset using Dask with specified dtypes
df_dask = dd.read_csv(file_pattern, assume_missing=True, on_bad_lines="skip", dtype=dtype_spec)

# End timing
end_time = time.time()
print(f"Dask Load Data - Wall Time: {end_time - start_time} seconds")

# Check missing values in the dataset
missing_values = df_dask.isnull().sum().compute()
print("Missing Values:\n", missing_values)

# Drop columns with a high percentage of missing values (e.g., >50%)
threshold = 0.5 * len(df_dask)
columns_to_drop = missing_values[missing_values > threshold].index.tolist()
df_dask = df_dask.drop(columns=columns_to_drop)

# Compute basic statistics
statistics = df_dask.describe(include='all').compute()
print("Basic Statistics:\n", statistics)

# Optionally, save the cleaned and processed dataset to a file
output_path = '/content/nyc-parking-tickets-cleaned.csv'
df_dask.to_csv(output_path, single_file=True, index=False)
print(f"Processed dataset saved to {output_path}")

Dask Load Data - Wall Time: 0.04765605926513672 seconds
Missing Values:
 Summons Number                           0
Plate ID                                 6
Registration State                       0
Plate Type                               0
Issue Date                               0
Violation Code                           0
Vehicle Body Type                      217
Vehicle Make                           401
Issuing Agency                           0
Street Code1                             0
Street Code2                             0
Street Code3                             0
Vehicle Expiration Date                  0
Violation Location                   10964
Violation Precinct                       0
Issuer Precinct                          0
Issuer Code                              0
Issuer Command                       10890
Issuer Squad                         10895
Violation Time                           0
Time First Observed                  54910
Violation County        

In [27]:
import time

# Measure wall time for calculating missing values
start_time = time.time()

# Calculate missing values
missing_values = df_dask.isnull().sum().compute()

end_time = time.time()
print(f"Missing Values - Wall Time: {end_time - start_time} seconds")
print("Missing Values:\n", missing_values)

Missing Values - Wall Time: 0.6294870376586914 seconds
Missing Values:
 Summons Number                           0
Plate ID                                 6
Registration State                       0
Plate Type                               0
Issue Date                               0
Violation Code                           0
Vehicle Body Type                      217
Vehicle Make                           401
Issuing Agency                           0
Street Code1                             0
Street Code2                             0
Street Code3                             0
Vehicle Expiration Date                  0
Violation Location                   10964
Violation Precinct                       0
Issuer Precinct                          0
Issuer Code                              0
Issuer Command                       10890
Issuer Squad                         10895
Violation Time                           0
Violation County                       205
Violation In Front Of Or 

In [29]:
import time

# Columns to drop that still exist in the DataFrame
columns_to_drop = ['House Number']  # Remove 'Time First Observed' if already dropped

# Measure wall time for dropping columns
start_time = time.time()

# Drop the columns
df_dask_dropped = df_dask.drop(columns=columns_to_drop)

end_time = time.time()
print(f"Dropping Columns - Wall Time: {end_time - start_time} seconds")

# Optionally, compute the result to finalize the operation
df_dask_dropped.compute()

Dropping Columns - Wall Time: 0.019963502883911133 seconds


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
0,5.092469e+09,GZH7067,NY,PAS,07/10/2016,7.0,SUBN,TOYOT,V,0.0,...,1111.0,D,<NA>,<NA>,<NA>,GY,2001.0,0.0,<NA>,FAILURE TO STOP AT RED LIGHT
1,5.092452e+09,GZH7067,NY,PAS,07/08/2016,7.0,SUBN,TOYOT,V,0.0,...,1111.0,D,<NA>,<NA>,<NA>,GY,2001.0,0.0,<NA>,FAILURE TO STOP AT RED LIGHT
2,4.006265e+09,FZX9232,NY,PAS,08/23/2016,5.0,SUBN,FORD,V,0.0,...,1111.0,C,<NA>,<NA>,<NA>,BK,2004.0,0.0,<NA>,BUS LANE VIOLATION
3,8.478630e+09,66623ME,NY,COM,06/14/2017,47.0,REFG,MITSU,T,10610.0,...,408.0,l2,Y,0700A,0700P,WH,2007.0,0.0,04,47-Double PKG-Midtown
4,7.868300e+09,37033JV,NY,COM,11/21/2016,69.0,DELV,INTER,T,10510.0,...,408.0,h1,Y,0700A,0700P,WHITE,2007.0,0.0,31 6,69-Failure to Disp Muni Recpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59603,8.352516e+09,NYRPL8,NY,SPO,09/07/2016,21.0,4DSD,HYUND,T,9430.0,...,408.0,d1,Y,1130A,0100P,BL,2015.0,0.0,49 3,21-No Parking (street clean)
59604,4.629733e+09,HJN2753,NY,PAS,01/20/2017,36.0,SUBN,NISSA,V,0.0,...,1180.0,B,<NA>,<NA>,<NA>,GY,2015.0,0.0,<NA>,PHTO SCHOOL ZN SPEED VIOLATION
59605,7.035981e+09,2AY4004,MD,PAS,07/07/2016,14.0,4DSD,TOYOT,T,52390.0,...,408.0,c,YYYYYYY,<NA>,<NA>,BLACK,0.0,0.0,05-A,14-No Standing
59606,1.411087e+09,GWV1149,NY,PAS,09/02/2016,71.0,SDN,ME/BE,P,59420.0,...,408.0,E2,BBBBBBB,ALL,ALL,GREY,2009.0,0.0,<NA>,<NA>


In [30]:
import time

# Measure wall time for basic statistics
start_time = time.time()

# Compute basic statistics
statistics = df_dask.describe().compute()

end_time = time.time()
print(f"Basic Statistics - Wall Time: {end_time - start_time} seconds")

# Print the statistics
print(statistics)

Basic Statistics - Wall Time: 0.9789533615112305 seconds
       Summons Number  Violation Code  Street Code1  Street Code2  \
count    5.960800e+04    59608.000000  59608.000000  59608.000000   
mean     6.859667e+09       34.826886  24085.768722  20434.984063   
std      2.304381e+09       19.374050  22661.650866  21857.334346   
min      1.131601e+09        1.000000      0.000000      0.000000   
25%      5.093227e+09       20.000000   5780.000000      0.000000   
50%      8.129096e+09       36.000000  18230.000000  14080.000000   
75%      8.486803e+09       40.000000  35490.000000  33697.500000   
max      8.585450e+09       99.000000  97940.000000  98260.000000   

       Street Code3  Vehicle Expiration Date  Violation Location  \
count  59608.000000             5.960800e+04        48644.000000   
mean   20499.578547             2.481589e+07           55.556842   
std    22023.090382             2.617651e+07           37.895600   
min        0.000000             0.000000e+00     